<a href="https://colab.research.google.com/github/a2kiti/colab_web_demo/blob/master/vgg16_web_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.models import load_model
import numpy as np
import os
from scipy.misc import imread, imsave, imresize

In [0]:
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
model = VGG16(weights='imagenet')
#Executing once to avoid errors
preds = model.predict(preprocess_input(np.zeros((1,224,224,3))))

In [0]:
def imageResize(img, h, w):
    ih, iw = img.shape[0], img.shape[1]
    if ih >= iw:
        dst = np.ones([ih, ih, 3], np.uint8) * 255
        dst[:, (ih - iw)//2:(ih - iw)//2 + iw] = img
    else:
        dst = np.ones([iw, iw, 3], np.uint8) * 255
        dst[(iw - ih)//2:(iw - ih)//2 + ih, :] = img
        
    dst = imresize(dst, (h, w))
    return dst        


In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

In [0]:
#Writing to html file on colab

html = """

<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8" />
</head>
<body>

<form action="/post" method="post" enctype="multipart/form-data">
    <p><input type="file" name="uploadFile"/></p>
    <p><input type="submit" value="send"/></p>
</form>

{% if ulr_Image %}

    <p>Image</p>
    <p><img src="{{ ulr_Image }}"></p>
    <p>Result</p>
    <p>{{ result_class }}</p>
    <p>{{ result_score }}</p>

{% endif %}

</body>
</html>

"""

#Saving html file
if not os.path.isdir( "templates" ):
    os.makedirs( "templates" )
with open("templates/index.html", mode='w') as f:
    f.write(html)

#Making tmp folder to use temporarily stored
if not os.path.isdir( "tmp" ):
    os.makedirs( "tmp" )

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

from flask import Flask, render_template, send_from_directory, request, redirect, url_for

app = Flask(__name__)

@app.route("/")
def index():
    return render_template( "index.html" )

@app.route('/post', methods=['POST'])
def post():
    try:
        uploadFile = request.files['uploadFile']
    except:
        return redirect(url_for('index'))
    
    uploadFile.save("tmp/" + uploadFile.filename)
    
    img = imread("tmp/" + uploadFile.filename)
    if img.shape[-1] == 4:
        img = img[:,:,0:3]
    img = imageResize(img, 224, 224)
    fileName = str(np.random.randint(1000)) + ".jpg"
    imsave("tmp/" + fileName, img)
    ulr_Image = "tmp/" + fileName
    
    img = np.expand_dims(img, axis=0)
    print(img.shape)
    preds = model.predict(preprocess_input(img))
    result = decode_predictions(preds, top=1)[0][0]
    result_class = "Class: " + result[1]
    result_score = "Score: " + str(result[2])
    
    return render_template( "index.html",
                                        ulr_Image = ulr_Image,
                                        result_class = result_class,
                                        result_score = result_score)

@app.route('/tmp/<filename>')
def uploaded_file(filename):
    return send_from_directory("./tmp", filename)

if __name__ == '__main__':
    app.run(port=6006)